In [1]:
# Script to load FAQ file for use in RAG runtime

from google.cloud import bigquery

# Load CSV to BigQuery

# --- Configuration ---
gcs_uri = "gs://labs.roitraining.com/aurora-bay-faqs/aurora-bay-faqs.csv"
dataset_id = "rag_challenge"
table_id = "aurora_bay_faqs"

# GCP Details
project_id = "qwiklabs-gcp-03-28c3125acb2b"
client = bigquery.Client(project=project_id)

# Initialize the BigQuery client
client = bigquery.Client()

# Construct the full table ID string
table_id_full = f"{client.project}.{dataset_id}.{table_id}"

# --- Load Job Configuration ---
job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,  # Assuming your CSV has a header row
    autodetect=True,      # Let BigQuery automatically infer the schema
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
)

# --- Start the Load Job ---
print(f"Starting load job for '{gcs_uri}' into '{table_id_full}'...")

load_job = client.load_table_from_uri(
    gcs_uri,
    table_id_full,
    job_config=job_config,
)  # API request

# Wait for the job to complete
load_job.result()

print("Load job finished.")

# Load files
try:
    destination_table = client.get_table(table_id_full)
    print(f"Loaded {destination_table.num_rows} rows into {table_id_full}.")
except Exception as e:
    print(f"Could not get table details after load: {e}")

Starting load job for 'gs://labs.roitraining.com/aurora-bay-faqs/aurora-bay-faqs.csv' into 'qwiklabs-gcp-03-28c3125acb2b.rag_challenge.aurora_bay_faqs'...
Load job finished.
Loaded 50 rows into qwiklabs-gcp-03-28c3125acb2b.rag_challenge.aurora_bay_faqs.


In [ ]:
# SQL scripts used to generate embeddings in BQ (This block contains SQL queries directly run in BQ)

#SQL Used in BQ

#0: Note that I did have to run an alter statement to make the two column headers question & answer

#1:
#CREATE OR REPLACE MODEL `rag_challenge.embeddings_model`
#REMOTE WITH CONNECTION `us.vertex-rag-connection`
#OPTIONS (ENDPOINT = 'text-embedding-005');

#2:
#CREATE OR REPLACE TABLE `rag_challenge.auror_bay_faqs_with_embeddings` AS
#SELECT *
#FROM ML.GENERATE_EMBEDDING(
#    MODEL `rag_challenge.embeddings_model`,
#    (SELECT question, answer, concat(question, answer) AS content FROM `qwiklabs-gcp-03-28c3125acb2b.rag_challenge.aurora_bay_faqs`)
#);

In [26]:
# RAG Runtime Script

import pandas as pd
from google.cloud import bigquery
from google import genai
from google.genai import types
import base64

def generate_response(user_input, sys_instruction):

  client = genai.Client(
      vertexai=True,
      project="qwiklabs-gcp-03-28c3125acb2b",
      location="us-central1",
  )

  model = "gemini-2.0-flash-001"

  contents = [
    types.Content(
      role="user",
      parts=[
        types.Part.from_text(text = user_input)
      ]
    )
  ]

  generate_content_config = types.GenerateContentConfig(
    temperature = 1,
    top_p = 0.95,
    max_output_tokens = 8192,
    response_modalities = ["TEXT"],
    speech_config = types.SpeechConfig(
      voice_config = types.VoiceConfig(
        prebuilt_voice_config = types.PrebuiltVoiceConfig(
          voice_name = "zephyr"
        )
      ),
    ),
    system_instruction=[types.Part.from_text(text=sys_instruction)],
  )

  output_string = ""

  for chunk in client.models.generate_content_stream(
    model = model,
    contents = contents,
    config = generate_content_config,
    ):
    #print(chunk.text, end="")
    output_string = output_string + chunk.text

  return output_string


def main():

  # Inputs
  user_input_query = 'How many people live in Aurora Bay?'

  # GCP Details
  project_id = "qwiklabs-gcp-03-28c3125acb2b"
  client = bigquery.Client(project=project_id)

  # Initialize the BigQuery client
  client = bigquery.Client()

  # Step 1: Get the Top 5 Matching Results from BQ

  # Setup query to get top 5 results from BQ via similarity search, with the Vertex AI model connection generating the embedding of the user input question
  query = f"""
  SELECT query.query, base.question, base.answer
  FROM VECTOR_SEARCH(
    TABLE `qwiklabs-gcp-03-28c3125acb2b.rag_challenge.auror_bay_faqs_with_embeddings`, 'ml_generate_embedding_result',
    (
    SELECT text_embedding, content AS query
    FROM ML.GENERATE_TEXT_EMBEDDING(
    MODEL `qwiklabs-gcp-03-28c3125acb2b.rag_challenge.embeddings_model`,
    (SELECT @query_text AS content))
    ),
    top_k => 5, options => '{{"fraction_lists_to_search": 0.01}}')
  """

  # Configuration of the user input parameter
  job_config = bigquery.QueryJobConfig(
    query_parameters = [
        bigquery.ScalarQueryParameter("query_text", "STRING", user_input_query),
    ]
  )

  top5_results_df = client.query(query, job_config=job_config).to_dataframe()

  # Step 2: Construct Prompt from the Top 5 Matching Results from BQ
  # Construct Prompt
  prompt_qnas = "Please Use the Reference Question And Answers to answer the user's query: \n"

  for row in top5_results_df.itertuples():
      prompt_qnas = prompt_qnas + f"Question: {row.question}, Answer: {row.answer}\n"

  # Step 3: Add back in the original user question
  final_prompt = prompt_qnas + "\nUser Query: \n" + user_input_query

  #print(final_prompt)

  # Step 4: Get Response from Gemini

  sys_instructions = """You are a helpful FAQ agent who helps answer user questions on the topic of Aurora Bay. An example set of questions will be provided for additional context."""
  final_rag_results = generate_response(final_prompt, sys_instructions)

  print(final_prompt)
  print("\n")
  print(final_rag_results)

if __name__ == "__main__":
    main()


Please Use the Reference Question And Answers to answer the user's query: 
Question: What is the population of Aurora Bay?, Answer: Aurora Bay has a population of approximately 3,200 residents, although it can fluctuate seasonally due to temporary fishing and tourism workforces.
Question: What is the average temperature range in Aurora Bay?, Answer: Winters average between 10°F to 25°F, while summers are milder, around 50°F to 65°F. Temperatures can vary with coastal weather patterns.
Question: Who is the current mayor of Aurora Bay?, Answer: The current mayor is Linda Greenwood, elected in 2021 for a four-year term.
Question: Is there a local hospital in Aurora Bay?, Answer: Yes. The Aurora Bay Community Hospital offers emergency and routine medical services. It’s located on North Aurora Boulevard.
Question: What are the primary industries in Aurora Bay?, Answer: The primary industries include commercial fishing, tourism, and small-scale logging in the nearby forests.

User Query: 
Ho